In [1]:
import pandas as pd
import os

In [2]:
%pwd

'/home/omar/Desktop/End-to-end-Machine-Learning-Project-with-MLflow/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/omar/Desktop/End-to-end-Machine-Learning-Project-with-MLflow'

In [5]:
df = pd.read_csv('artifacts/data_ingestion/predictive_maintenance.csv')
df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   UDI                      10000 non-null  int64  
 1   Product ID               10000 non-null  object 
 2   Type                     10000 non-null  object 
 3   Air temperature [K]      10000 non-null  float64
 4   Process temperature [K]  10000 non-null  float64
 5   Rotational speed [rpm]   10000 non-null  int64  
 6   Torque [Nm]              10000 non-null  float64
 7   Tool wear [min]          10000 non-null  int64  
 8   Target                   10000 non-null  int64  
 9   Failure Type             10000 non-null  object 
dtypes: float64(3), int64(4), object(3)
memory usage: 781.4+ KB


In [7]:
df.rename(columns={'Target': 'Machine failure'}, inplace=True)

In [8]:
df.drop(['UDI', 'Product ID'], axis=1, inplace=True)
df.head()

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,Failure Type
0,M,298.1,308.6,1551,42.8,0,0,No Failure
1,L,298.2,308.7,1408,46.3,3,0,No Failure
2,L,298.1,308.5,1498,49.4,5,0,No Failure
3,L,298.2,308.6,1433,39.5,7,0,No Failure
4,L,298.2,308.7,1408,40.0,9,0,No Failure


kelvin to Celsius

In [9]:
df['Air temperature [c]'] = df['Air temperature [K]'] - 273.15
df['Process temperature [c]'] = df['Process temperature [K]'] - 273.15
df.drop(['Air temperature [K]', 'Process temperature [K]'], axis=1, inplace=True)
df.head()

,Type,Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,Failure Type,Air temperature [c],Process temperature [c]
0,M,1551,42.8,0,0,No Failure,24.95,35.45
1,L,1408,46.3,3,0,No Failure,25.05,35.55
2,L,1498,49.4,5,0,No Failure,24.95,35.35
3,L,1433,39.5,7,0,No Failure,25.05,35.45
4,L,1408,40.0,9,0,No Failure,25.05,35.55


In [10]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=[['L', 'M', 'H']])

df['Type'] = encoder.fit_transform(df[['Type']])
df.head()

,Type,Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,Failure Type,Air temperature [c],Process temperature [c]
0,1.0,1551,42.8,0,0,No Failure,24.95,35.45
1,0.0,1408,46.3,3,0,No Failure,25.05,35.55
2,0.0,1498,49.4,5,0,No Failure,24.95,35.35
3,0.0,1433,39.5,7,0,No Failure,25.05,35.45
4,0.0,1408,40.0,9,0,No Failure,25.05,35.55


In [11]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

df['type_of_failure'] = encoder.fit_transform(df['Failure Type'])
df.head()

,Type,Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,Failure Type,Air temperature [c],Process temperature [c],type_of_failure
0,1.0,1551,42.8,0,0,No Failure,24.95,35.45,1
1,0.0,1408,46.3,3,0,No Failure,25.05,35.55,1
2,0.0,1498,49.4,5,0,No Failure,24.95,35.35,1
3,0.0,1433,39.5,7,0,No Failure,25.05,35.45,1
4,0.0,1408,40.0,9,0,No Failure,25.05,35.55,1


In [12]:
encoder.classes_

array(['Heat Dissipation Failure', 'No Failure', 'Overstrain Failure',
       'Power Failure', 'Random Failures', 'Tool Wear Failure'],
      dtype=object)

In [13]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

col_to_scale = ['Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]', 'Air temperature [c]', 'Process temperature [c]']

df_scaled = scaler.fit_transform(df[col_to_scale])
df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = col_to_scale

df.drop(col_to_scale, axis=1, inplace=True)

df_scaled = pd.concat([df, df_scaled], axis=1)
df_scaled.head()

,Type,Machine failure,Failure Type,type_of_failure,Rotational speed [rpm],Torque [Nm],Tool wear [min],Air temperature [c],Process temperature [c]
0,1.0,0,No Failure,1,0.222934,0.535714,0.000000,0.304348,0.358025
1,0.0,0,No Failure,1,0.139697,0.583791,0.011858,0.315217,0.370370
2,0.0,0,No Failure,1,0.192084,0.626374,0.019763,0.304348,0.345679
3,0.0,0,No Failure,1,0.154249,0.490385,0.027668,0.315217,0.358025
4,0.0,0,No Failure,1,0.139697,0.497253,0.035573,0.315217,0.370370


In [14]:
X = df_scaled.drop('type_of_failure', axis=1)
y = df_scaled['type_of_failure']

In [15]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(sampling_strategy='auto')

X_resampled, y_resampled = ros.fit_resample(X, y)

df_sampled = pd.concat([X_resampled, y_resampled], axis=1)

In [16]:
df_sampled.shape

(57912, 9)

In [17]:
df_sampled['type_of_failure'].value_counts()

type_of_failure
1    9652
3    9652
5    9652
2    9652
4    9652
0    9652
Name: count, dtype: int64

In [18]:
df_sampled.head()


,Type,Machine failure,Failure Type,Rotational speed [rpm],Torque [Nm],Tool wear [min],Air temperature [c],Process temperature [c],type_of_failure
0,1.0,0,No Failure,0.222934,0.535714,0.000000,0.304348,0.358025,1
1,0.0,0,No Failure,0.139697,0.583791,0.011858,0.315217,0.370370,1
2,0.0,0,No Failure,0.192084,0.626374,0.019763,0.304348,0.345679,1
3,0.0,0,No Failure,0.154249,0.490385,0.027668,0.315217,0.358025,1
4,0.0,0,No Failure,0.139697,0.497253,0.035573,0.315217,0.370370,1


In [19]:
df_sampled.value_counts('type_of_failure')

type_of_failure
0    9652
1    9652
2    9652
3    9652
4    9652
5    9652
Name: count, dtype: int64

In [20]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataProcessingConfig:
    root_dir: Path 
    data_path: Path

In [21]:
from src.ML.constants import *
from src.ML.utils.common import read_yaml, create_directories

In [22]:
class ConfigurationManager:
    def __init__(self,
                config_filepath=CONFIG_FILE_PATH,
                params_filepath=PARAMS_FILE_PATH,
                schema_filepath=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_processing_config(self)-> DataProcessingConfig:
        config=self.config.data_processing

        create_directories([config.root_dir])

        data_processing_config = DataProcessingConfig(
            root_dir =config.root_dir,
            data_path=config.data_path
        )
        
        return data_processing_config 


In [23]:
import os
from src.ML import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

In [24]:
class DataProcessing:
    def __init__(self, config: DataProcessingConfig):
        self.config = config
        self.df = None  
    
    def load_data(self):
        """Load data from CSV file."""
        self.df = pd.read_csv(self.config.data_path)
        logger.info("Data loaded successfully")

    def rename_and_drop_columns(self):
        """Rename and drop specified columns."""
        self.df.rename(columns={'Target': 'Machine failure'}, inplace=True)
        self.df.drop(['UDI', 'Product ID'], axis=1, inplace=True)
        rename_dict = {
            'Air temperature [K]': 'Air_temperature_C',
            'Process temperature [K]': 'Process_temperature_C',
            'Rotational speed [rpm]': 'Rotational_Speed_RPM',
            'Torque [Nm]': 'Torque_Nm',
            'Tool wear [min]': 'Tool_Wear_min',
        }
        self.df.rename(columns=rename_dict, inplace=True)
        print(self.df.columns)
        logger.info("Renamed and dropped columns")

    def convert_temperature(self):
        """Convert temperature from Kelvin to Celsius and drop original columns."""
        self.df['Air_temperature_C'] = self.df['Air_temperature_C'] - 273.15
        self.df['Process_temperature_C'] = self.df['Process_temperature_C'] - 273.15
        logger.info("Converted temperatures to Celsius and dropped original columns")

    def encode_features(self):
        """Encode categorical features with OrdinalEncoder and LabelEncoder."""
        ordinal_encoder = OrdinalEncoder(categories=[['L', 'M', 'H']])
        self.df['Type'] = ordinal_encoder.fit_transform(self.df[['Type']])
        self.df['Type'] = pd.to_numeric(self.df['Type'], errors='coerce')
        label_encoder = LabelEncoder()
        self.df['Failure Type'] = label_encoder.fit_transform(self.df['Failure Type'])
        self.df['Failure Type'] = pd.to_numeric(self.df['Failure Type'], errors='coerce')

        logger.info("Encoded categorical features")

    def scale_features(self):
        """Scale specified numerical features with MinMaxScaler."""
        col_to_scale = ['Rotational_Speed_RPM', 'Torque_Nm', 'Tool_Wear_min', 'Air_temperature_C', 'Process_temperature_C']
        scaler = MinMaxScaler()
        self.df[col_to_scale] = scaler.fit_transform(self.df[col_to_scale])
        logger.info("Scaled numerical features")

    def balance_data(self):
        """Balance the dataset by undersampling 'No Failure' and oversampling other classes."""
        # Separate features and target variable
        X = self.df.drop('Failure Type', axis=1)
        y = self.df['Failure Type']

        # Undersample the "No Failure" class
        rus = RandomUnderSampler(sampling_strategy={'No Failure': 2500})
        X_under, y_under = rus.fit_resample(X, y)

        # Now, oversample the other classes to 1000
        ros = RandomOverSampler(sampling_strategy={
            'Heat Dissipation Failure': 250,
            'Power Failure': 230,
            'Overstrain Failure': 160,
            'Tool Wear Failure': 100,
            'Random Failures': 40
        })
        
        # Apply oversampling on the undersampled data
        X_balanced, y_balanced = ros.fit_resample(X_under, y_under)

        # Combine balanced features and target variable back into a DataFrame
        self.df = pd.concat([X_balanced, y_balanced], axis=1)

        logger.info("Balanced the dataset")

    def train_test_split(self):
        train, test = train_test_split(self.df, test_size=0.2, random_state=42)
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)
        logger.info("Data split into training and test sets")
        logger.info(f"Train shape: {train.shape}, Test shape: {test.shape}")

In [25]:
try:
    config = ConfigurationManager()
    data_processing_config = config.get_data_processing_config()
    data_processing= DataProcessing(config=data_processing_config)
    data_processing.load_data()
    data_processing.rename_and_drop_columns()
    data_processing.convert_temperature()
    data_processing.scale_features()
    data_processing.balance_data()
    data_processing.encode_features()
    data_processing.train_test_split()
except Exception as e:
    raise e

[2024-10-28 18:11:45,153: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-10-28 18:11:45,173: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-28 18:11:45,184: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-10-28 18:11:45,190: INFO: common: created directory at: artifacts]
[2024-10-28 18:11:45,200: INFO: common: created directory at: artifacts/data_processing]
[2024-10-28 18:11:45,298: INFO: 808294505: Data loaded successfully]
Index(['Type', 'Air_temperature_C', 'Process_temperature_C',
       'Rotational_Speed_RPM', 'Torque_Nm', 'Tool_Wear_min', 'Machine failure',
       'Failure Type'],
      dtype='object')
[2024-10-28 18:11:45,314: INFO: 808294505: Renamed and dropped columns]
[2024-10-28 18:11:45,338: INFO: 808294505: Converted temperatures to Celsius and dropped original columns]
[2024-10-28 18:11:45,378: INFO: 808294505: Scaled numerical features]
[2024-10-28 18:11:45,446: INFO: 808294505: Balanced the dataset]
[2024-10-